In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PointNet(nn.Module):
    def __init__(self, num_classes):
        super(PointNet, self).__init__()
        self.num_classes = num_classes
        
        # 입력점을 처리하는 MLP (Multi-layer Perceptron) 모듈을 정의합니다.
        self.input_mlp = nn.Sequential(
            nn.Conv1d(3, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )
        
        # 입력점에 대한 전역 특징 추출 모듈을 정의합니다.
        self.global_feature_extractor = nn.Sequential(
            nn.Conv1d(64, 128, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 1024, 1),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.MaxPool1d(1024)
        )
        
        # 분류를 위한 MLP 모듈을 정의합니다.
        self.classifier_mlp = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, self.num_classes)
        )

    def forward(self, x):
        # 입력점에 대한 MLP 모듈을 적용합니다.
        x = self.input_mlp(x)
        
        # 전역 특징 추출 모듈을 적용합니다.
        global_features = self.global_feature_extractor(x)
        
        # 전역 특징을 MLP 모듈에 적용하여 분류 결과를 출력합니다.
        output = self.classifier_mlp(global_features.view(-1, 1024))
        return output


nn.Conv1d는 1차원 합성곱(Convolution) 연산을 수행하는 모듈입니다. PointNet에서는 입력점의 좌표 정보를 각각 x, y, z 좌표로 나누어 각각을 1차원으로 처리하며, 각 점에 대한 정보를 모두 포함하는 입력을 위해 3채널을 사용합니다. nn.BatchNorm1d는 1차원 입력에 대한 Batch Normalization을 수행하는 모듈이며, nn.ReLU는 ReLU 활성화 함수를 적용합니다. nn.MaxPool1d는 1차원 Max Pooling 연산을 수행하는 모듈입니다.

모델의 입력은 N개의 점군입니다. 각 점은 3차원의 좌표값으로 표현되며, 이 점들은 PyTorch Tensor로 변환됩니다. 따라서, 모델의 forward 함수에서는 입력 Tensor x를 input_mlp 모듈을 통해 처리하여 각 점에 대한 특성을 추출합니다. 추출된 특성은 global_feature_extractor 모듈을 통해 전역 특징 벡터로 변환됩니다. 이 벡터는 classifier_mlp 모듈을 통해 클래스별로 분류됩니다.

마지막으로, 모델을 사용할 때는 다음과 같이 사용할 수 있습니다.

In [ ]:
model = PointNet(num_classes=10) # 분류할 클래스의 개수를 입력합니다.
input_points = torch.randn(4, 3, 1024) # 크기가 (4, 3, 1024)인 텐서를 입력합니다.
output = model(input_points) # 분류 결과를 출력합니다.

위의 코드에서 input_points는 3차원 좌표로 표현된 1024개의 점으로 이루어진 4개의 점군입니다. model 객체를 생성한 후, input_points를 모델의 forward 함수의 인자로 전달하여 분류 결과인 output을 얻을 수 있습니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

from pointnet import PointNet

# 하이퍼파라미터 설정
num_epochs = 100
batch_size = 32
learning_rate = 0.001

# GPU 사용 여부 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 데이터셋 클래스 정의
class ModelNetDataset(data.Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        # 데이터 로드 및 전처리 수행
        # 여기에서는 간단하게 로드만 하도록 하겠습니다.
        # 로드한 데이터는 N x 3 크기의 Tensor로 전환합니다.
        point_cloud = torch.load(self.file_list[index]).to(device)
        
        # 라벨 추출
        label = int(self.file_list[index].split('/')[-2])
        
        return point_cloud, label

# 데이터 로더 생성
train_file_list = [f"./ModelNet40/train/{i}/{j}" for i in range(40) for j in os.listdir(f"./ModelNet40/train/{i}")]
train_dataset = ModelNetDataset(train_file_list)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 모델 생성
model = PointNet(num_classes=40).to(device)

# 손실 함수와 최적화 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # 입력 데이터를 GPU에 올리기
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 역전파 단계 전 초기화
        optimizer.zero_grad()
        
        # 순전파 단계
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 역전파 단계
        loss.backward()
        optimizer.step()
        
        # 통계 출력
        running_loss += loss.item()
        if i % 10 == 9: # 10 배치마다 출력
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0


PointNet 모델을 사용하여 분류 문제를 학습하는 코드를 제공해 드리겠습니다. 이 코드는 ModelNet40 데이터셋을 사용하여 3D 모델을 분류하는 문제를 학습합니다.

위 코드에서는 ModelNet40 데이터셋을 사용하여 PointNet 모델을 학습하는 코드입니다. ModelNetDataset 클래스는 파일 리스트에서 3D 모델 데이터를 로드하고 전처리하는 역할을 합니다. train_loader는 DataLoader 객체를 생성하여 학습에 사용할 데이터를 배치 단위로 로드합니다.

학습 전, 모델은 to(device) 메소드를 사용하여 GPU로 전송합니다. 손실 함수로는 Cross Entropy Loss를 사용하고, 최적화 함수로는 Adam을 사용합니다. 학습은 for문을 사용하여 반복적으로 실행됩니다. 각 epoch마다 train_loader에서 배치 단위로 데이터를 로드하고, 모델의 출력과 정답 라벨을 사용하여 손실 함수를 계산하고, 역전파 단계를 수행하여 모델의 가중치를 갱신합니다.

학습 중간에 손실 함수의 값을 출력하여 학습 과정을 모니터링할 수 있습니다. 학습이 완료되면, 모델의 성능을 평가하고 테스트 데이터에 대한 분류 결과를 확인할 수 있습니다.